In [ ]:
#installing the libraries
pip install azureml-sdk

In [ ]:
pip install Workspace

In [ ]:
pip install Dataset

In [ ]:
#importing libraries
import pandas as pd
import numpy as np

from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split


In [ ]:
ws=Workspace.from_config()

In [ ]:
#Reading the data
Diabetes_data=pd.read_csv('diabetes6.csv')
Diabetes_data

In [ ]:
Diabetes_data.head()

In [ ]:
Diabetes_data.columns

In [ ]:
x_train, x_test = train_test_split(Diabetes_data, test_size=0.2)

In [ ]:
automl_settings={
    "iteration_timeout_minutes":2,
    "experiment_timeout_minutes":15,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "n_cross_validations":5,
}

In [ ]:
automl_config = AutoMLConfig(task='classification',
         debug_log='automl_errors.log',
         training_data=x_train,
         label_column_name="Outcome",
         **automl_settings)


In [ ]:
experiment = Experiment(ws, "Diabetes_experiment")

In [ ]:
run=experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, model = run.get_output()

In [ ]:
RunDetails(run).show()

In [ ]:
model_name = best_run.properties["model_name"]

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model 
from  azureml.core.environment import Environment


In [ ]:
best_run.download_file("outputs/scoring_file_v_1_0_0.py", "inference/score.py")

In [ ]:
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, "myenv.yml") 
env = Environment.from_conda_specification(name="myenv",file_path="myenv.yml") 

inference_config = InferenceConfig(entry_scripts="inference/score.py", environment=env)
aciconfig Aciwebservice.deploy_configuration(cpu_cores=1, memory_gb=1, descriptions="diabetes classification")
service = Model.deploy(ws, "automl-diabetes", [registered_name], inference_config,aciconfig)
service.wait_for_deployment(True)
